Horizontal plot of annual mean (2005) SST which only shows values over the 10% of the total area characterized by the highest temperatures.

In [1]:
%matplotlib notebook 
import numpy as np
import pandas as pd
import numpy.ma as ma
import netCDF4 as nc4
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.colors as colors
from matplotlib import colors, ticker, cm
import matplotlib.patches as mpatches
import cartopy as cart
from mpl_toolkits.basemap import Basemap
import xarray as xrr
from numpy.ma import masked_where
import copy

In [ ]:
#
fyd=nc4.Dataset('/group_workspaces/jasmin2/nemo/vol1/ORCA0083-N006/domain/mesh_hgr.nc','r',format='NETCDF4')
e1t=fyd.variables['e1t'][0,1373:1493,3865:4045]
e2t=fyd.variables['e2t'][0,1373:1493,3865:4045]    
areas = e1t*e2t

fyd=nc4.Dataset('/group_workspaces/jasmin2/nemo/vol1/ORCA0083-N006/domain/mask.nc','r',format='NETCDF4')
mask = fyd.variables['tmask'][0,0,1373:1493,3865:4045]

m_area=areas*mask
area_tot=np.sum(m_area,dtype=np.float32)
area_tot_10 = area_tot*0.1
area_tot_10

In [28]:
#load in monthly SST data from 2005:
m_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
sst=np.zeros((120,180,12))
va=0
for m in m_list:
    fyd=nc4.Dataset('/group_workspaces/jasmin2/nemo/vol1/ORCA0083-N006/means/2005/ORCA0083-N06_2005m'+str(m)+'T.nc','r',format='NETCDF4')
    sst_=fyd.variables['sst'][0,1373:1493,3865:4045]
    sst[:,:,va]=sst_
    va=va+1
    
sst[np.abs(sst) > 3000.] = np.nan 
sst[sst == 0.] = np.nan
#mask = masked_where(np.abs(mask) > 0, mask)
#mask[mask == 0] = 1

In [29]:
latS=fyd.variables['nav_lat'][1373:1493,3865:4045]       
lonS=fyd.variables['nav_lon'][1373:1493,3865:4045] 

In [ ]:
sst_m=np.nanmean(sst,axis=2)
sst_or=np.sort(sst_m,axis=None)
sst_or1 = sst_or[::-1]
sst_or2=sst_or1[np.isfinite(sst_or1)]
sst_or2
sst_or3=sst_or2[:]

In [ ]:
s = e1t[np.where(sst_m == np.nanmax(sst_m))]
dd = e2t[np.where(sst_m == np.nanmax(sst_m))]  
s*dd

In [ ]:
ar = e1t[np.where(sst_m == np.nanmax(sst_m))]*e2t[np.where(sst_m == np.nanmax(sst_m))]
#ar
sst_m1 = copy.deepcopy(sst_m)

In [33]:
masimo = np.zeros(14297)
area_m = np.zeros(14297)
i=0

while (np.sum(area_m) < area_tot_10):
        masimo[i] = np.nanmax(sst_m1)
        area_mm = e1t[np.where(sst_m1 == np.nanmax(sst_m1))]*e2t[np.where(sst_m1 == np.nanmax(sst_m1))]
        area_mm = area_mm[0]
        #np.append(area_m,area_mm)
        area_m[i]=area_mm
        sst_m1[sst_m1 == np.nanmax(sst_m1)]=0
        i=i+1

SST = masimo[masimo !=0]

In [ ]:
#np.sum(area_m)
#area_tot_10

In [67]:
#Masks
#Continent and islands
fyd=nc4.Dataset('/group_workspaces/jasmin2/nemo/vol1/ORCA0083-N006/domain/mask.nc','r',format='NETCDF4')
maskc=fyd.variables['tmask'][0,0,1373:1493,3865:4045]
maskc = masked_where(np.abs(maskc) > 0, maskc)
maskc[maskc == 0] = 1

#Temperatures
mask = np.ma.masked_less(sst_m, 27.831504185994465)
#np.where(SST == np.min(SST))
#np.min(SST)

In [ ]:
#2D annual mean SST surface field 

plt.figure(figsize=(7, 5))
plt.plot(111)

#ticks=[26.5,26.75, 27, 27.25, 27.5, 27.75, 28, 28.25]
low=27.83
high=28.1
ran=0.01

P1 = plt.contourf(lonS,latS,sst_m,np.arange(low, high, ran),extend='both',
                  cmap=plt.cm.spectral)
#plt.contourf(lonS,latS,mask,vmin=27, vmax=30,cmap=plt.cm.Greys)
plt.contourf(lonS,latS,mask,cmap=plt.cm.spectral)


plt.title('Annual Mean SST - 2005')
plt.ylim((-10,0))
plt.xlim((35,50))
plt.xlabel('E Longitude')
plt.ylabel('S Latitude')

In [ ]:
#2D annual mean SST surface field 
plt.figure(figsize=(7, 5))
plt.plot(111)

ticks=[26.25, 26.5, 26.75, 27, 27.25, 27.5, 27.75, 28]
low=26.4
high=28.1
ran=0.025

P1 = plt.contourf(lonS,latS,sst_m,np.arange(low, high, ran),extend='both',
                  cmap=plt.cm.spectral)


plt.contourf(lonS,latS,maskc,vmin=0,vmax=2,cmap=plt.cm.Greys)

cbar = plt.colorbar(P1)
#plt.contour(lonS,latS,mask,colors='k',levels=[10], linewidths=0.5)
cbar.set_ticks(ticks)
cbar.set_label('$^\circ$C',rotation=0,labelpad=14)
plt.title('Annual Mean SST - 2005')
plt.ylim((-10,0))
plt.xlim((35,50))
plt.xlabel('E Longitude')
plt.ylabel('S Latitude')

In [ ]:
#2D annual mean SST surface field 

plt.figure(figsize=(7, 5))
plt.plot(111)

ticks=[27.85, 27.9,27.95,28,28.05]
low=27.83
high=28.1
ran=0.025

P1 = plt.contourf(lonS,latS,sst_m,np.arange(np.min(SST),np.max(SST),0.01),
                  extend='both',cmap=plt.cm.spectral)

cbar = plt.colorbar(P1)
cbar.set_ticks(ticks)
cbar.set_label('$^\circ$C',rotation=0,labelpad=14)
plt.title('10% of the area with highest annual Mean SST - 2005',fontsize=10)
plt.ylim((-10,0))
plt.xlim((35,50))
plt.xlabel('E Longitude')
plt.ylabel('S Latitude')